In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Input
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# =========================
# CONFIG
# =========================
BATCH_SIZE = 64
EPOCHS = 30

# =========================
# LOAD PREPROCESSED DATA
# =========================
X = np.load("X_cnn.npy")   # shape: (N, 1024, 2, 1)
y = np.load("y.npy")

print("Loaded:", X.shape, y.shape)

# Detect number of classes automatically
NUM_CLASSES = len(np.unique(y))
print("Detected classes:", NUM_CLASSES)

# Shuffle entire dataset
X, y = shuffle(X, y, random_state=42)

# =========================
# SPLIT (70 / 15 / 15)
# =========================
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

print("Train:", X_train.shape)
print("Val:  ", X_val.shape)
print("Test: ", X_test.shape)

# =========================
# NORMALIZE (based on train only)
# =========================
mean = np.mean(X_train)
std = np.std(X_train) + 1e-8

X_train = (X_train - mean) / std
X_val   = (X_val   - mean) / std
X_test  = (X_test  - mean) / std

# =========================
# ONE-HOT
# =========================
y_train = to_categorical(y_train, NUM_CLASSES)
y_val   = to_categorical(y_val, NUM_CLASSES)
y_test  = to_categorical(y_test, NUM_CLASSES)

# =========================
# CNN MODEL (2D Conv)
# =========================
model = Sequential([
    Input(shape=(1024, 2, 1)),

    Conv2D(32, kernel_size=(5, 2), activation="relu"),
    MaxPooling2D(pool_size=(2, 1)),

    Conv2D(64, kernel_size=(5, 1), activation="relu"),
    MaxPooling2D(pool_size=(2, 1)),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.3),
    Dense(NUM_CLASSES, activation="softmax")
])

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

# =========================
# TRAIN
# =========================
early_stop = EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True,
    callbacks=[early_stop],
    verbose=1
)

test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print("\nCNN Test Accuracy:", test_acc)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# =========================
# PREDICTIONS
# =========================
y_true = np.argmax(y_test, axis=1)
y_pred = np.argmax(model.predict(X_test, verbose=0), axis=1)

NUM_CLASSES = y_test.shape[1]
class_names = [f"TX {i}" for i in range(NUM_CLASSES)]

# =========================
# CLASSIFICATION REPORT
# =========================
print("\n==============================")
print(" CNN CLASSIFICATION REPORT")
print("==============================\n")

print(classification_report(
    y_true,
    y_pred,
    target_names=class_names,
    digits=4
))

# =========================
# CONFUSION MATRIX (RAW)
# =========================
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(5,5))
disp = ConfusionMatrixDisplay(
    confusion_matrix=cm,
    display_labels=class_names
)
disp.plot(cmap="Blues", values_format="d")
plt.title("CNN Confusion Matrix (Counts)")
plt.grid(False)
plt.tight_layout()
plt.show()

# =========================
# CONFUSION MATRIX (NORMALIZED)
# =========================
cm_norm = cm.astype(float) / cm.sum(axis=1, keepdims=True)

plt.figure(figsize=(5,5))
disp_norm = ConfusionMatrixDisplay(
    confusion_matrix=cm_norm,
    display_labels=class_names
)
disp_norm.plot(cmap="Blues", values_format=".2f")
plt.title("CNN Confusion Matrix (Normalized)")
plt.grid(False)
plt.tight_layout()
plt.show()

# =========================
# PER-CLASS ACCURACY
# =========================
print("\nPer-class accuracy:")
for c, name in enumerate(class_names):
    idx = y_true == c
    acc = np.mean(y_pred[idx] == c)
    print(f"{name}: {acc:.4f}")

# =========================
# OVERALL ACCURACY
# =========================
overall_acc = np.mean(y_true == y_pred)
print(f"\nOverall Test Accuracy: {overall_acc:.4f}")

# =========================
# CORRECT vs INCORRECT IQ SCATTER (CNN)
# =========================
# Same visualization style as DNN/RNN
# =========================
# CNN IQ SCATTER (channels_last)
# =========================
cls = 0

correct_idx = np.where((y_true == cls) & (y_pred == cls))[0][:200]
wrong_idx   = np.where((y_true == cls) & (y_pred != cls))[0][:200]

plt.figure(figsize=(6,6))

if len(correct_idx) > 0:
    I_correct = X_test[correct_idx][:, :, 0, 0].flatten()
    Q_correct = X_test[correct_idx][:, :, 1, 0].flatten()
    plt.scatter(I_correct, Q_correct, s=1, alpha=0.3, label="Correct")

if len(wrong_idx) > 0:
    I_wrong = X_test[wrong_idx][:, :, 0, 0].flatten()
    Q_wrong = X_test[wrong_idx][:, :, 1, 0].flatten()
    plt.scatter(I_wrong, Q_wrong, s=1, alpha=0.5, label="Misclassified")

plt.xlabel("In-phase (I)")
plt.ylabel("Quadrature (Q)")
plt.title(f"CNN IQ Scatter – TX {cls}")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()